## Data Retrieval and Cleaning

In [1]:
import requests
from pprint import pprint
import yfinance as yf
import pandas as pd
import time
import numpy as np
import regex

In [2]:
def sectordata():
    import yfinance as yf
    from pprint import pprint
    import pandas as pd

    #using the yahoo finance package to get data for the dow jones industrial adverage and various sector index funds
    
    data = yf.download('^DJI','2020-01-22','2021-10-11')   #retrieves and stores dow jones open values
    data = data.iloc[:,0]
    data.to_csv("DOW_price.csv")
    sector_data = pd.DataFrame(data,columns = ["Open"])
    
    i=0    #loop retrieves data for the selected index funds; i used to initialize pandas df for other values to be added to
    lisq = [("XLE","Energy"),("XLF","Financial Services"),("XLK","Technology"),("XLRE","Real Estate"),("XLC","Communication Services"),("XLU","Utilities"),("XLP","Consumer Staples"),("XLV","Healthcare"),("XLB","Basic Materials")]
    for tick, name in lisq:
        i += 1
        data = yf.download('{}'.format(tick),'2020-01-22','2021-10-11')
        data = data.iloc[:,0]
        if i == 1:
            sector_data["Energy"] = pd.DataFrame(data)
        else:
            sector_data["{}".format(name)] = data
    sector_data.to_csv("Sector_Prices2.csv")
    pprint(sector_data)
sectordata()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
                    Open     Energy  Financial Services  Technology  \
Date                                                                  
2020-01-21  29269.050781  58.790001           30.879999   96.820000   
2020-01-22  29263.630859  57.950001           30.870001   97.690002   
2020-01-23  29111.019531  

In [3]:
import requests
import pandas as pd
import datetime

#using the guardian's api to find articles mentioning covid or coronavirus

#finding number of pages retuned from the api to use in loop later
res = requests.get("https://content.guardianapis.com/search?q=covid%20OR%20caronavirus&api-key=ad1362b7-3b61-4cc4-8fde-f1c0e57c3cd9")
data = res.json()
pages = data["response"]["pages"]


used = [] #initializing
dj = 0
outdic = {}
s = requests.Session()  #creating a session for the many get requests in the loop
start = datetime.datetime.now()  #used to get runtime; testing improvement from adding a requests session
for i in range(pages):  #querying each page of the api and creating a dictionary with dates as keys and the # of articles as values
    gurls = "https://content.guardianapis.com/search?page={}&q=covid%20OR%20caronavirus&api-key=ad1362b7-3b61-4cc4-8fde-f1c0e57c3cd9".format(i+1)
    resp = s.get(gurls)
    resp = resp.json()
    pagedata = resp["response"]["results"]
    for each in pagedata:
        datex = each["webPublicationDate"]
        date = datex[:10]
        if date in used:
            outdic[date] += 1
        else:
            used += [date]
            outdic[date] = 1

pd.DataFrame.from_dict(data=outdic, orient='index').to_csv("api_dic_sessiontest.csv", header=False)
finish = datetime.datetime.now() - start
print("Runtime:")
print(finish)
print(outdic)

Runtime:
0:11:07.482567
{'2022-04-26': 9, '2022-02-21': 31, '2022-02-06': 14, '2022-03-25': 16, '2022-05-04': 3, '2022-05-05': 7, '2022-04-17': 11, '2022-04-05': 12, '2022-02-19': 22, '2022-03-08': 7, '2022-01-17': 31, '2022-04-10': 3, '2022-04-19': 23, '2022-04-18': 11, '2022-04-08': 11, '2022-02-20': 24, '2022-04-13': 22, '2022-03-17': 16, '2022-03-13': 7, '2022-05-03': 9, '2022-05-06': 11, '2022-04-01': 11, '2022-04-27': 8, '2021-12-30': 36, '2022-04-11': 11, '2022-03-22': 11, '2022-04-29': 13, '2022-04-20': 14, '2022-01-26': 25, '2022-04-24': 11, '2022-03-05': 4, '2022-03-30': 20, '2022-04-22': 12, '2022-02-16': 23, '2022-03-03': 4, '2022-03-12': 7, '2022-03-02': 20, '2022-03-27': 9, '2022-04-21': 18, '2021-12-03': 27, '2022-02-08': 22, '2022-04-15': 5, '2022-01-09': 16, '2022-03-20': 14, '2022-04-12': 18, '2021-12-08': 39, '2022-03-16': 12, '2022-04-06': 11, '2022-04-28': 9, '2022-02-27': 11, '2022-04-04': 20, '2022-03-18': 9, '2022-03-10': 19, '2022-04-03': 6, '2022-05-02': 6, '2

In [4]:
import pandas as pd
with open("covid_data.csv") as fcov:
    dd = pd.read_csv(fcov,delimiter=',')
    ddd = dd.fillna(0)
    ddd = ddd.iloc[:, :12]   #getting necessary values
    colnames = ddd.columns
    colnames = colnames[2:]
    for name in colnames:
        for i in ddd.index:
            if ddd.loc[i, name] < 0: #replacing negative numbers with zero as negative cases/deaths doesn't make sense
                ddd.loc[i, name] = 0

    ddd["submission_date"] = pd.to_datetime(ddd["submission_date"])
    data = ddd.sort_values(by = "submission_date")  #combining data from different states on the same date into a national total
    data = data.groupby(["submission_date"]).sum()
    
    data.to_csv("COV_dat.csv")
    print(data.head())

                 tot_cases  conf_cases  prob_cases  new_case  pnew_case  \
submission_date                                                           
2020-01-01               0         0.0         0.0         0        0.0   
2020-01-02               0         0.0         0.0         0        0.0   
2020-01-03               0         0.0         0.0         0        0.0   
2020-01-04               0         0.0         0.0         0        0.0   
2020-01-05               0         0.0         0.0         0        0.0   

                 tot_death  conf_death  prob_death  new_death  pnew_death  
submission_date                                                            
2020-01-01               0         0.0         0.0          0         0.0  
2020-01-02               0         0.0         0.0          0         0.0  
2020-01-03               0         0.0         0.0          0         0.0  
2020-01-04               0         0.0         0.0          0         0.0  
2020-01-05        